In [1]:
# Useful starting lines
%matplotlib inline
import datetime
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from implementations import*

In [3]:


DATA_TRAIN_PATH = '../data/train.csv/train.csv' # TODO: download train data and supply path here 
y, x, ids = load_csv_data(DATA_TRAIN_PATH)

# Pre-processing


In [4]:
selector = np.all(x != -999.0, axis=1)
x_clean = x[selector]
y_clean = y[selector]

print(x.shape)
print(x_clean.shape)
    

(250000, 30)
(68114, 30)


In [5]:
def standardize_NAN(tX):
    tX_nan = tX.copy()
    for i in range(tX.shape[0]):
        for j in range(tX.shape[1]):
            if (tX_nan[i,j] == -999.0):
                tX_nan[i,j] = np.nan
    return (standardize(tX_nan))


# Tout les nans (correspondant a des valeurs non connues) sont remplacés par la moyenne de la colonnes
def replace_mean(tX_nan):
    means_cols = np.nanmean(tX_nan,axis=1)
    for row in range(0,tX_nan.shape[0]):
        for col in range(0,tX_nan.shape[1]):
            if np.isnan(tX_nan[row,col]):
                tX_nan[row,col]=means_cols[col]
    return (tX_nan)

x_nan, mean_x_nan, std_x_nan = standardize_NAN(x)
x_nan = replace_mean(x_nan)

def get_ind_percentiles(tX, tX_clean, i, percentile):
    arguments = []
    a = np.percentile(tX_clean[:,i],percentile)
    tX_perc = tX.copy()
    arguments = np.argwhere(tX_perc[tX[:,i] > round(a, 2)])
    return list(set(arguments[:,0]))

def remove_rows_by_percentiles(tX,tX_clean):
    args = []
    for i in range(tX.shape[1]):
        args= args+get_ind_percentiles(tX,tX_clean,i,99.97)
    flat_list = [item for item in args]
    mylist = list(set(flat_list))
    return mylist

92.56765498468643


In [6]:
arg = remove_rows_by_percentiles(x,x_clean)
x_perc = np.delete(x, arg, axis=0)
y_perc = np.delete(y, arg, axis=0)

selected_columns0 = [1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 29]
selected_columns1 = [1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 29]
selected_columns_ideal = [0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 29]

def selected_non_nan_columns(x):
    x_selected = np.zeros((len(x), len(selected_columns0)))
    for i in range(len(x)):
        s = np.take(x[i], indices=selected_columns0, axis=0)
        x_selected[i] = s
    return x_selected

In [7]:
x_sel = selected_non_nan_columns(x_perc)
x_sel.shape

(234948, 19)

In [8]:
v = x_sel != -999
v.all()

True

In [9]:
x_s = x_sel
y_s = y_perc

# Ridge Regression

In [10]:
x_poly = build_poly(x_s, 2)
lambda_ = 0.1
w, loss = ridge_regression_s(y_s, x_poly, lambda_)
x_poly[:1]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  6.60090000e+01,  1.49462000e+02,
         1.36136000e+02,  2.80100000e+00,  3.46930000e+01,
         2.72747000e+02,  3.24000000e-01,  4.86000000e-01,
         8.10770000e+01, -1.30900000e+00,  1.24600000e+00,
         2.63040000e+01,  1.03900000e+00, -2.82000000e-01,
         5.96570000e+01,  1.68800000e+00,  3.42194000e+02,
         1.00000000e+00,  1.65366000e+02,  4.35718808e+03,
         2.23388894e+04,  1.85330105e+04,  7.84560100e+00,
         1.20360425e+03,  7.43909260e+04,  1.04976000e-01,
         2.36196000e-01,  6.57347993e+03,  1.71348100e+00,
         1.55251600e+00,  6.91900416e+02,  1.07952100e+0

In [14]:
#NOT RUN
indices = build_k_indices(y_s, 4, seed=1)
x_test = x_s[indices[0]]
x_train = np.delete(x_s, [indices[0]], axis=0)
x_train.shape
loss_tr, loss_te = cross_validation_rr(y_s, x_s, k_indices=indices, k=1, lambda_=0.1, degree=3)

In [11]:

def cross_validation_demo_RR(x,y,K):
    seed = 1
    degree = 4
    k_fold = K
    lambdas = np.logspace(-4, 0, 30)
    
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    x_test = x[k_indices[0]]
    x_train = np.delete(x, [indices[0]], axis=0)
    
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    
    for i in range(len(lambdas)):
        l = lambdas[i]
        avg_err_tr = 0
        avg_err_te = 0
        for k in range(k_fold):
            err = cross_validation_rr(y, x, k_indices, k, l, degree)
            avg_err_tr += err[0]
            avg_err_te += err[1]
        rmse_tr.append(np.sqrt(2 * avg_err_tr / k_fold))
        rmse_te.append(np.sqrt(2 * avg_err_te / k_fold))
    cross_validation_visualization(lambdas, rmse_tr, rmse_te)
    
    min_err_index = 0
    for i in range(1, len(rmse_te)):
        if rmse_te[i] < rmse_te[min_err_index]:
            min_err_index = i
            
    print('Best lambda is: {0}'.format(lambdas[min_err_index]))       
    
cross_validation_demo_RR(x_s,y_s,K)

degree_opt = 4
lambda_opt = 0.0006723357536499335
x_poly = build_poly(x_s, degree_opt)
w_rr_opt, loss_tr = ridge_regression_s(y_s, x_poly, lambda_opt)
print("Training set mse: {}".format(loss_tr))

#Training Accuracy
y_predicted = predict_labels(w_rr_opt, x_poly)
accuracy = []
accuracy.append((list(y_clean == y_predicted).count(True))/len(y_clean))
print("accuracy = {val}".format(val=accuracy))

NameError: name 'K' is not defined

# 1.Least squares gradient descent

In [ ]:

def cross_validation_LS_GD_demo(x_LS,y_LS,K): 
    #Adding constant term
    tX_LS = np.c_[np.ones((y_LS.shape[0], 1)), x_LS]

    max_iters = 100
    gammas = np.logspace(-4,0,20)


    # Initialization
    w_initial = np.zeros(tX_LS.shape[1])

    list_tX_LS = np.split(tX_LS,K)
    list_y_LS = np.split(y_LS,K)

    gen_opt_w=[]
    gen_mse =[]

    #gamma selection
    for ind, gamma in enumerate(gammas):
        weights=[]
        mse_errors = []
        #K-fold crossvalidation
        for ind, tX_bloc in enumerate(list_tX_LS):
            tX_test = tX_bloc
            y_test = list_y_LS[ind]
            tX_train= list_tX_LS[:ind] + list_tX_LS[ind+1:]
            tX_train= np.concatenate(tX_train)
            y_train= list_y_LS[:ind] + list_y_LS[ind+1:]
            y_train=np.concatenate(y_train)
        
            mse, opt_w = least_squares_GD(y_train, tX_train, w_initial, max_iters, gamma)
            mse_errors.append(compute_mse(y_test, tX_test,opt_w))
            weights.append(opt_w)
        gen_mse.append(np.mean(mse_errors))
        gen_opt_w.append(np.mean(weights, axis=0))
    optimal_gamma_LS_GD = gammas[np.nanargmin(gen_mse)]
    optimal_weights_LS_GD = gen_opt_w[np.nanargmin(gen_mse)]
    mse_LS_GD = np.nanmin(gen_mse)
    print(" gamma={l:.3f},mse={mse:.3f}".format(mse = mse_LS_GD, l = optimal_gamma_LS_GD))

    #Training Accuracy
    y_predicted = predict_labels(optimal_weights_LS_GD, tX_LS)
    accuracy = []
    accuracy.append((list(y_LS == y_predicted).count(True))/len(y_LS))
    print(accuracy)
    #return accuracy,optimal_gamma_LS_GD, optimal_wights_LS_GD,mse_LS_GD

#With tX_CLEAN : accuracy = 0.6823, mse= 0.825, gamma = 0.0055
#With tX_NAN : accuracy= 0.705, mse= 0.737, gamma = 0.089
#With x_te_s: accuracy = 0.702, mse = 0.759, gamma = 0.144


In [ ]:
#With tX and y_LS no corrupted
print("dataset1")
x_LS, m_X,s = standardize(x_clean.copy())
x_LS = x_LS[0:68110]
y_LS = y_clean[0:68110].copy()
cross_validation_LS_GD_demo(x_LS,y_LS,5)

#With nan value replaced
print("dataset2")
x_LS=x_nan.copy()
y_LS=y.copy()
cross_validation_LS_GD_demo(x_LS,y_LS,5)
#With x_te
print("dataset3")
x_LS =x_s
y_LS=y_s
cross_validation_LS_GD_demo(x_LS,y_LS,4)

# Least square SDG

We can alter the gamma and the batch size

In [12]:
def cross_validation_LS_SGD_demo(x_LS,y_LS,K):
    #Adding constant term
    tX_LS = np.c_[np.ones((y_LS.shape[0], 1)), x_LS]

    max_iters = 50
    max_batch_size = 32
    gammas = np.logspace(-4,0,20)
    batch_sizes = np.array([2,4,6,8])

    # Initialization
    w_initial = np.zeros(tX_LS.shape[1])
    list_tX_LS = np.split(tX_LS,K)
    list_y_LS = np.split(y_LS,K)


    result_mse =[]
    result_opt_w=[]
    result_gamma=[]
    for ind_batch,batch_size in enumerate(batch_sizes):  
        result_mse_gamma = []
        result_opt_w_gamma = []
        for ind_gamma,gamma in enumerate(gammas):
            mse_errors=[]
            weights=[]
            #K-fold crossvalidation
            for ind, tX_bloc in enumerate(list_tX_LS):
                tX_test = tX_bloc
                y_test = list_y_LS[ind]
                tX_train= list_tX_LS[:ind] + list_tX_LS[ind+1:]
                tX_train= np.concatenate(tX_train)
                y_train= list_y_LS[:ind] + list_y_LS[ind+1:]
                y_train=np.concatenate(y_train)
        
                sgd_mse, opt_w = least_squares_SGD(y_train, tX_train, w_initial, batch_size, max_iters, gamma)
                mse_errors.append(compute_mse(y_test, tX_test,opt_w))
                weights.append(opt_w)
    
            result_mse_gamma.append(np.mean(mse_errors))
            result_opt_w_gamma.append(np.mean(weights,axis=0))
        result_mse.append(np.min(result_mse_gamma))
        result_gamma.append(gammas[np.argmin(result_mse_gamma)])
        result_opt_w.append(result_opt_w_gamma[np.argmin(result_mse_gamma)])

    print(" gamma={l:.3f}, batch={b:.2f}, mse={mse:.3f}".format(mse = np.nanmin(result_mse), l =result_gamma[np.nanargmin(result_mse)], b=np.nanargmin(result_mse)))

    optimal_weights_LS_SGD = result_opt_w[np.nanargmin(result_mse)]
    

    #Training Accuracy
    y_predicted = predict_labels(optimal_weights_LS_SGD, tX_LS)
    accuracy = []
    accuracy.append((list(y_LS == y_predicted).count(True))/len(y_LS))
    print(accuracy)

#With tX_CLEAN : accuracy = , mse= , gamma = , batch=
#With tX_NAN : accuracy= , mse= , gamma = ,batch=



In [14]:
#With tX and y_LS no corrupted
print("dataset1")
x_LS, m_X,s = standardize(x_clean.copy())
x_LS = x_LS[0:68110]
y_LS = y_clean[0:68110].copy()
#cross_validation_LS_SGD_demo(x_LS,y_LS,5)
#With nan value replaced
print("dataset2")
x_LS=x_nan.copy()
y_LS=y.copy()
cross_validation_LS_SGD_demo(x_LS,y_LS,5)
#With x_te
print("dataset3")
x_LS =x_s
y_LS=y_s
cross_validation_LS_SGD_demo(x_LS,y_LS,4)

dataset1
134.10663740969926
dataset2


KeyboardInterrupt: 

# Logistic regression (not regularized)

In [17]:
def cross_validation_LR_demo(x_LR,y_LR,K):

    #Adding constant term
    tX_LR = np.c_[np.ones((y_LR.shape[0], 1)), x_LR]
    max_iters = 100
    gammas = np.logspace(-4,0,20)

    # Initialization
    w_initial = np.zeros(tX_LR.shape[1])

    list_tX_LR = np.split(tX_LR,K)
    list_y_LR = np.split(y_LR,K)

    gen_opt_w=[]
    gen_loss =[]

    #gamma selection
    for ind, gamma in enumerate(gammas):
        weights=[]
        loss_errors = []
        #K-fold crossvalidation
        for ind, tX_bloc in enumerate(list_tX_LR):
            tX_test = tX_bloc
            y_test = list_y_LR[ind]
            tX_train= list_tX_LR[:ind] + list_tX_LR[ind+1:]
            tX_train= np.concatenate(tX_train)
            y_train= list_y_LR[:ind] + list_y_LR[ind+1:]
            y_train=np.concatenate(y_train)
            loss, opt_w = logistic_regression(y_train,tX_train,w_initial, max_iters, gamma)
            loss_errors.append(calculate_loss_logistic_reg(y_test, tX_test,opt_w))
            weights.append(opt_w)
        gen_loss.append(np.mean(loss_errors))
        gen_opt_w.append(np.mean(weights, axis=0))


    optimal_gamma_LR = gammas[np.nanargmin(gen_loss)]
    optimal_weights_LR = gen_opt_w[np.nanargmin(gen_loss)]
    print(" gamma={l:.3f},loss={loss:.3f}".format(loss = np.min(gen_loss), l = optimal_gamma_LR))


     #Training Accuracy
    y_predicted = predict_labels(optimal_weights_LR, tX_LR)
    accuracy = []
    accuracy.append((list(y_LR == y_predicted).count(True))/len(y_LR))
    print(accuracy)

#With tX_CLEAN : accuracy = 0.6823, loss= , gamma = 
#With tX_NAN : accuracy= 0.34, loss= nan, gamma = 0.114
#With x_te_s : accuracy = 0.7015, loss = 16775, gamma = 0.144

In [18]:
#With tX and y_LS no corrupted
print("dataset1")
x_LS, m_X,s = standardize(x_clean.copy())
x_LS = x_LS[0:68110]
y_LS = y_clean[0:68110].copy()
cross_validation_LR_demo(x_LS,y_LS,5)

#With nan value replaced
print("dataset2")
x_LS=x_nan.copy()
y_LS=y.copy()
cross_validation_LR_demo(x_LS,y_LS,5)
#With x_te
print("dataset3")
x_LS =x_s
y_LS=y_s
cross_validation_LR_demo(x_LS,y_LS,4)

dataset1
134.10663740969926


C:\Users\emmal\OneDrive\Documents\Cours M1 S1\Machine Learning\ML_course-master\ML_course\projects\project1\scripts\implementations.py:20: RuntimeWarning: divide by zero encountered in log
  log = np.sum(y.T@np.log(o)+(1-y.T)@np.log(1-o))
C:\Users\emmal\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 gamma=0.055,loss=nan
[0.6823814417853472]
dataset2
 gamma=0.089,loss=nan
[0.704824]
dataset3


C:\Users\emmal\OneDrive\Documents\Cours M1 S1\Machine Learning\ML_course-master\ML_course\projects\project1\scripts\implementations.py:157: RuntimeWarning: invalid value encountered in subtract
  w = w-gamma*gradient


ValueError: All-NaN slice encountered

# Logistic regression regularized with lambda

In [19]:
def cross_validation_LRR_demo(x_LRR,y_LRR,K):
    #Adding constant term
    tX_LRR = np.c_[np.ones((y_LRR.shape[0], 1)), x_LRR]

    max_iters = 50
    lambdas = np.logspace(-4,0,10)
    gammas = np.logspace(-4,0,20)

    # Initialization
    w_initial = np.zeros(tX_LRR.shape[1])
    list_tX_LRR = np.split(tX_LRR,K)
    list_y_LRR = np.split(y_LRR,K)


    result_loss =[]
    result_opt_w=[]
    result_gamma=[]
    for ind,lambda_ in enumerate(lambdas):  
        result_loss_gamma = []
        result_opt_w_gamma = []
        for ind_gamma,gamma in enumerate(gammas):
            loss_errors=[]
            weights=[]
            #K-fold crossvalidation
            for ind, tX_bloc in enumerate(list_tX_LRR):
                tX_test = tX_bloc
                y_test = list_y_LRR[ind]
                tX_train= list_tX_LRR[:ind] + list_tX_LRR[ind+1:]
                tX_train= np.concatenate(tX_train)
                y_train= list_y_LRR[:ind] + list_y_LRR[ind+1:]
                y_train=np.concatenate(y_train)
        
                loss, opt_w = reg_logistic_regression(y_train,tX_train,lambda_,w_initial,max_iters,gamma)
                loss_errors.append(calculate_loss_logistic_reg(y_test, tX_test,opt_w))
                weights.append(opt_w)
    
            result_loss_gamma.append(np.mean(loss_errors))
            result_opt_w_gamma.append(np.mean(weights,axis=0))
        result_loss.append(np.min(result_loss_gamma))
        result_gamma.append(np.argmin(result_loss_gamma))
        result_opt_w.append(result_opt_w_gamma[np.argmin(result_loss_gamma)])

    print(np.min(result_loss))
    print(result_gamma[np.argmin(result_loss)])
    print(np.argmin(result_loss))
    print(" gamma={l:.3f}, batch={b:.0f}, mse={mse:.3f}".format(mse = np.min(result_loss), l =result_gamma[np.argmin(result_loss)], b=np.argmin(result_loss)))

    optimal_weights_LRR = result_opt_w[np.argmin(result_loss)]
    print(optimal_weights_LRR)

    #Training Accuracy
    y_predicted = predict_labels(optimal_weights_LRR, tX_LRR)
    accuracy = []
    accuracy.append((list(y_LRR == y_predicted).count(True))/len(y_LRR))
    print(accuracy)

In [20]:
#With tX and y_LS no corrupted
print("dataset1")
x_LS, m_X,s = standardize(x_clean.copy())
x_LS = x_LS[0:68110]
y_LS = y_clean[0:68110].copy()
cross_validation_LRR_demo(x_LS,y_LS,5)

#With nan value replaced
print("dataset2")
x_LS=x_nan.copy()
y_LS=y.copy()
cross_validation_LRR_demo(x_LS,y_LS,5)
#With x_te
print("dataset3")
x_LS =x_s
y_LS=y_s
cross_validation_LRR_demo(x_LS,y_LS,4)


dataset1
134.10663740969926


MemoryError: 

## Generate predictions and save ouput in csv format for submission:

In [26]:
DATA_TEST_PATH = '../data/test.csv/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [30]:
OUTPUT_PATH = './logisticRegression_x_te_s' # TODO: fill in desired name of output file for submission
tX_test = np.c_[np.ones((tX_test.shape[0], 1)), tX_test]
tX_test = selected_non_nan_columns(tX_test)
y_pred = predict_labels(optimal_weights_LR, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

ValueError: shapes (568238,32) and (20,) not aligned: 32 (dim 1) != 20 (dim 0)